In [39]:
from langchain_groq import ChatGroq
from groq_api_key import key

In [40]:
llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    api_key=key,
    temperature=0
)

In [41]:
from langchain_community.document_loaders import WebBaseLoader

In [42]:
loader = WebBaseLoader('https://jobs.nike.com/job/R-44279?from=job%20search%20funnel')
page_data = loader.load().pop().page_content
print(page_data)

Apply for Director, Software Engineering

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsTechnologyDirector, Software EngineeringBeaverton, OregonBecome a Part of the NIKE, Inc. TeamNIK

In [43]:
from langchain_core.prompts import PromptTemplate

In [44]:
prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE:
    {page_data}
    ### INSTRUCTION:
    The scraped text is from the career's page of a website. Your job is to extract the job postings and return them in JSON format
    containing the following keys: 'role', 'experience', 'skills' and 'description'
    Only return the valid JSON
    VALID JSON NO PREAMBLE
    JUST THE JSON
    DONT ADD ```
    """
)

In [45]:
chain_extract = prompt_extract | llm
response = chain_extract.invoke(input={'page_data':page_data})

In [46]:
print(response.content)

[
  {
    "role": "Director, Software Engineering",
    "experience": "At least 12 years of overall experience as a technology leader with at least 7 of these years in a people management role building and leading effective, innovative teams in complex transformations with strong customer success mindset",
    "skills": [
      "Bachelor's degree in Computer Science, Engineering, Information Systems, or similar field",
      "At least 7 years as an individual contributor developing custom software and distributed platforms using common languages, cloud platforms, runtimes, and developer toolsets across broad technology stacks",
      "At least 7 years as an individual contributor developing highly available, highly scalable platforms using common languages and technologies including but not limited to Java, Node.js, Golang, React, or JavaScript",
      "At least 7 years of professional experience with Cloud Native technologies on platforms like AWS or other major cloud providers",
    

In [47]:
prompt_email = PromptTemplate.from_template(
    """ 
    JOB DESCRIPTION
    {job_description}

    ### INSTRUCTION:
    You are Teslim, a business development executive at TestDev Inc. TestDev Inc. is an AI & SOftware company
    that brings about the seamless integration of business processes through automated tools.
    Over our experience, we have empowered numerous enterprises with tailored solutions such as process optimization,
    cost reduction, and heightened overall eficiency.
    Your job is to write a cold email to the client regarding the job mentioned above and talk about fulfilling their needs.
    Also add the most relevant ones from the following links to showcase TestDev Inc. portfolio {link_list}.
    Remember, you are Teslim, CEO of TestDev Inc.
    Do not provide a preamble.
    ### EMAIL (NO PREAMBLE)
    """
)

In [57]:
chain_email = prompt_email | llm
response = chain_email.invoke(input={
    'job_description':str(job),
    'link_list':links
})

In [58]:
print(response.content)

Subject: Expert Software Engineering Solutions for Nike's Multi-Year Simplification and Modernization Journey

Dear Hiring Manager,

I came across the job description for the Director, Software Engineering role at Nike, and I am excited to introduce TestDev Inc. as a potential partner to help fulfill your software engineering needs. With over a decade of experience in empowering enterprises with tailored solutions, we are confident that our expertise aligns with Nike's multi-year platform roadmaps and modern industry trends.

Our team of experts has a deep understanding of software engineering practices, particularly in distributed systems and cloud-native architectures. We have a proven track record of delivering efficient, reusable, secure, and resilient technology solutions that meet the highest standards of quality and scalability.

Our portfolio showcases our expertise in various technologies, including:

* Cloud Native technologies on platforms like AWS (https://example.com/devop

In [48]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_response = json_parser.parse(response.content)

In [49]:
import pandas as pd

cv = pd.read_csv('my_portfolio.csv')

In [50]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name='portfolio')

if not collection.count():
    for _, row in cv.iterrows():
        collection.add(
            documents=row['Techstack'],
            metadatas={
                "links":row['Links']
            },
            ids=[str(uuid.uuid4())]
        )

In [51]:
links = collection.query(
    query_texts=['Experience in Python', 'Expertise in React'],
    n_results=2
).get('metadatas')

In [52]:
job = json_response[0]
skills = job['skills']

In [53]:
links = collection.query(
    query_texts=skills,
    n_results=2
).get('metadatas')

In [54]:
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/xamarin-portfolio'},
  {'links': 'https://example.com/devops-portfolio'}],
 [{'links': 'https://example.com/react-portfolio'},
  {'links': 'https://example.com/full-stack-js-portfolio'}],
 [{'links': 'https://example.com/xamarin-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/kotlin-backend-portfolio'}],
 [{'links': 'https://example.com/xamarin-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/full-stack-js-portfolio'},
  {'links': 'https://example.com/devops-portfolio'}]]

In [55]:
skills

["Bachelor's degree in Computer Science, Engineering, Information Systems, or similar field",
 'At least 7 years as an individual contributor developing custom software and distributed platforms using common languages, cloud platforms, runtimes, and developer toolsets across broad technology stacks',
 'At least 7 years as an individual contributor developing highly available, highly scalable platforms using common languages and technologies including but not limited to Java, Node.js, Golang, React, or JavaScript',
 'At least 7 years of professional experience with Cloud Native technologies on platforms like AWS or other major cloud providers',
 'At least 4+ years of professional experience with container runtimes, such as Kubernetes, deployment tools like Terraform, and configuration management solutions like Ansible',
 'Deep foundational knowledge of software engineering practices, particularly on distributed systems and cloud native architectures',
 'Excellent verbal and written comm